In [22]:

import os

In [23]:
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
import pandas as pd
import datetime
warnings.filterwarnings('ignore')

df = pd.read_csv("./Sales_Data/data.csv")
#df2 = pd.read_excel(url)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31/7/12,31/7/12,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,5/2/13,7/2/13,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17/10/13,18/10/13,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28/1/13,30/1/13,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,5/11/13,6/11/13,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [24]:
# Step 0: Load, filter, clean, and aggregate the data at the customer level

df = df[(df.Segment == 'Consumer') & (df.Country == 'United States')]
# infer_datetime_format=True
df['Order Date'] =  pd.to_datetime(df['Order Date'], infer_datetime_format=True, format='%d%m%Y')

In [25]:
# divide it into year, month, day
df['Order Date'].max().date()

datetime.date(2014, 12, 31)

## Step 1: Create RFM Features for each customers

In [26]:
# df.groupby('A').agg(['min', 'max'])

df_RFM = df.groupby('Customer ID').agg({'Order Date': lambda y: (df['Order Date'].max().date() - y.max().date()).days,
                                        'Order ID': lambda y: len(y.unique()),  
                                        'Sales': lambda y: round(y.sum(),2)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
RB-19360,96,6,15117.34
AB-10105,41,10,14473.57
KL-16645,47,12,14175.23
SC-20095,349,9,14142.33
HL-15040,43,6,12873.30


## Step 2: To automate the segmentation we will use 80% quantile for Recency and Monetary

In [27]:
# We will use the 80% quantile for each feature
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM.head()


     Recency  Frequency  Monetary
0.8    267.4        8.0   4070.17


,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
RB-19360,96,6,15117.34,2,1,2
AB-10105,41,10,14473.57,2,2,2
KL-16645,47,12,14175.23,2,2,2
SC-20095,349,9,14142.33,1,2,2
HL-15040,43,6,12873.30,2,1,2


## Step 3: Calculate RFM score and sort customers

In [28]:
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM.head()

,Recency,Frequency,Monetary,R,F,M,RMScore
Customer ID,,,,,,,
RB-19360,96,6,15117.34,2,1,2,22
AB-10105,41,10,14473.57,2,2,2,22
KL-16645,47,12,14175.23,2,2,2,22
SC-20095,349,9,14142.33,1,2,2,21
HL-15040,43,6,12873.30,2,1,2,22


In [29]:
df_RFM = df_RFM.reset_index()
df_RFM.head()

,Customer ID,Recency,Frequency,Monetary,R,F,M,RMScore
0,RB-19360,96,6,15117.34,2,1,2,22
1,AB-10105,41,10,14473.57,2,2,2,22
2,KL-16645,47,12,14175.23,2,2,2,22
3,SC-20095,349,9,14142.33,1,2,2,21
4,HL-15040,43,6,12873.30,2,1,2,22


In [54]:
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'Customer ID': "count",
                                        'Frequency': lambda y: round(y.mean(),0), # the mean of each partiition after grouping
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM

,Customer ID,Frequency,Recency,R,M,Monetary
RMScore,,,,,,
11,70,5,472,1,1,1642.0
12,257,6,90,2,1,1866.0
21,12,7,422,1,2,7969.0
22,70,8,68,2,2,6734.0


In [38]:
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM


,Customer ID,Frequency,Recency,R,M,Monetary
RMScore,,,,,,
22,70,8,68,2,2,6734.0
21,12,7,422,1,2,7969.0
12,257,6,90,2,1,1866.0
11,70,5,472,1,1,1642.0


## Step 4: Visualize the Value Matrix and explore some key numbers

In [62]:
# 1) Average Monetary Matrix
df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_M

R,1,2
M,,
2,7969.0,6734.0
1,1642.0,1866.0


In [57]:
# 2) Number of Customer Matrix
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='Customer ID')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_C


R,1,2
M,,
2,12,70
1,70,257


In [60]:
# 3) Recency Matrix
df_RFM_R = df_RFM_SUM.pivot(index='M', columns='R', values='Recency')
df_RFM_R= df_RFM_R.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_R

R,1,2
M,,
2,422,68
1,472,90


In [ ]:
"""
Some takeaways / quick wins with very simple sales & marketing tactics examples?

There are few customers in the “Disengaged” bucket and they have an average revenue higher than the “Star” bucket. 
Since there are very few of them, it should be easy to partner with the business to understand what happened at the
customer level. Based on the analysis, there might be a simple quick win: reactivate few of them with a phone call 
or meeting to hopefully move them back to the “Star” bucket (e.g. engaged customers).

The average last order from the “Light” bucket is very old (more than 1 year vs. 60-70 days for ‘engaged’ customers). 
Launching a simple reactivation campaign with a coupon might be an initiative that could lead to some new orders and 
help some of these customers move to the “New” bucket (e.g. engaged customers).

Simple Tactics Examples
The notebook is available on Github. Thank you for reading my post if you enjoyed it, please clap. 
Feel free to contact me if you want to make simple or more complex RFM segmentations within your organization.
"""

